### Database works.
In this **notebook** we have a lot of good stuff:

In [1]:
%load_ext sql
import os, getpass, time, xuleta
from glob import glob

user = input("Username: ")
password = getpass.getpass('Enter your password: ')
database = input("Database: ")
connection_string = "postgresql://%s:%s@localhost/%s" %(user,password,database)
%sql $connection_string

Username: denis
Enter your password: ········
Database: drought


'Connected: denis@drought'

### INPUT PARAMETERS
Make sure the **table** matches the list provide in the NODATA DICTIONARY block

In [2]:
schema = "chirpsX"
table = "prec_abs"
imfolder = "/home/denis/Downloads/1ano/clipped/"

In [18]:
try:
    xuleta.renamedate(imfolder,oldf="%Y-%j",newf="%Y-%m-%d",wts=0)
    print('Filenames converted to Y-m-d')
except:
    print('filenames already in Y-m-d format')

Done!
Filenames converted to Y-m-d


In [4]:
t0 = time.time()
#avoiding cagadas when inputing the image folder
if imfolder[-1] != "/":
    imfolder = imfolder + "/"
# NODATA DICTIONARY
nodatas = {}
# Fill in the entries one by one
nodatas["gpp"] = 32767
nodatas["lai"] = 255
nodatas["albedo"] = 32767
nodatas["psn"] = 32767
nodatas["ndvi"] = -20000 #usar o convertido para int16, cheaper!
nodatas["lstd"] = 65535
nodatas["lstn"] = 65535
nodatas["vpd"] = -32768
nodatas["zscore_dec"] = -32768
nodatas["anom_dec"] = -32768
nodatas["prec_abs"] = -32768
nodatas["t04"] = -9999 #for ESI
nodatas["t12"] = -9999 #for ESI
nodatas["prec"] = -9999 #for raw precipitation
nodatas["et_myd"] = 32767 #for Modis ET
nodatas["et_mod"] = 32767 #for Modis ET
nodatas["mcd43a3_raw"] = 254
nodatas["mcd43a3_anom"] = 32767
nodatas["mcd15a2h_raw"] = 254
nodatas["mcd15a2h_anom"] = 32767
nodatas["mxd16a2_raw"] = 0
nodatas["mxd16a2_anom"] = 32767


#and set the nodata value to be used from now on
nodata = str(nodatas[table])
print("nodata value for %s is set to %s" %(table.upper(),nodata))

nodata value for PREC_ABS is set to -32768


### Creating Schema and table

It might not be necessary to create a schema.

In [5]:
SQL = ("CREATE SCHEMA %s;" %(schema))
%sql $SQL

Done.


[]

In [6]:
#CREATING TABLE
SQL = ("""
CREATE TABLE %s.%s(
rid serial NOT NULL,
rast RASTER,
filename text,
acquisition date,
CONSTRAINT %s_%s_pkey
PRIMARY KEY (rid));
CREATE INDEX %s_%s_wkb_rast_idx
ON %s.%s
USING GIST (ST_ConvexHull(rast)); """ 
       %(schema,table,schema,table,schema,table,schema,table))
%sql $SQL

Done.
Done.


[]

In [7]:
%%time
comment = table.upper() + " data for " + schema + "."
SQL = ("""
COMMENT ON TABLE %s.%s 
IS '%s'
"""
    %(schema,table,comment))
%sql $SQL

Done.
CPU times: user 2.07 ms, sys: 850 µs, total: 2.92 ms
Wall time: 82.4 ms


### Importing images to the database

You can watch the verbose in the terminal to make sure that things are going ok.

In [8]:
importraster = "raster2pgsql -a -C -F -M -s 4326 -t 100x100 -N %s %s*.tif %s.%s | psql -p 5432 -d drought -U denis" %(nodata,imfolder,schema,table)
print(importraster)

raster2pgsql -a -C -F -M -s 4326 -t 100x100 -N -32768 /home/denis/Downloads/1ano/clipped/*.tif chirpsX.prec_abs | psql -p 5432 -d drought -U denis


In [9]:
%%time
os.system(importraster)

CPU times: user 1.59 ms, sys: 4.7 ms, total: 6.29 ms
Wall time: 5.14 s


0

#### Creating date index from filename

In [11]:
%%time
SQL = ("""
UPDATE %s.%s
    SET acquisition = date(
    substring(filename FROM 1 FOR 4) || '-' ||
    substring(filename FROM 6 FOR 2) || '-' ||
    substring(filename FROM 9 FOR 2))::date;
    
CREATE INDEX %s_acquisition_idx
    ON %s.%s (acquisition);
"""
      %(schema,table,table,schema,table))
#execute query
%sql $SQL

DataError: (psycopg2.DataError) invalid input syntax for type date: "2002-01-ti"
 [SQL: "UPDATE chirpsX.prec_abs\n    SET acquisition = date(\n    substring(filename FROM 1 FOR 4) || '-' ||\n    substring(filename FROM 6 FOR 2) || '-' ||\n    substring(filename FROM 9 FOR 2))::date;"]

### Testing

In [26]:
point = [str(-92.891),str(42.436)]#US
point = [str(-38.8),str(-8.03)] #PB

dates = [('1980-10-29'),('2017-01-09')]

In [27]:
# POINT BASED TEST
SQL = ("""SELECT 
        rid, 
        acquisition,
        ST_Value(rast, ST_SetSRID(ST_MakePoint(%s,%s), 4326))*0.01 AS %s
    FROM %s.%s
         WHERE ST_Intersects(ST_SetSRID(ST_MakePoint(%s,%s), 4326), rast)
           AND acquisition BETWEEN '%s' AND '%s'
    ORDER BY acquisition;
"""
%(point[0],point[1],table,schema,table,point[0],point[1],dates[0],dates[1]))
print(SQL)
%sql $SQL

SELECT 
        rid, 
        acquisition,
        ST_Value(rast, ST_SetSRID(ST_MakePoint(-38.8,-8.03), 4326))*0.01 AS mxd16a2_raw
    FROM modis_ne2.mxd16a2_raw
         WHERE ST_Intersects(ST_SetSRID(ST_MakePoint(-38.8,-8.03), 4326), rast)
           AND acquisition BETWEEN '1980-10-29' AND '2017-01-09'
    ORDER BY acquisition;

663 rows affected.


rid,acquisition,mxd16a2_raw
578,2002-07-04,0.83
2021,2002-07-12,0.57
3464,2002-07-20,0.73
4907,2002-07-28,0.0
6350,2002-08-05,0.0
7793,2002-08-13,0.0
9236,2002-08-21,0.81
10679,2002-08-29,0.53
12122,2002-09-06,0.49
13565,2002-09-14,0.38


In [28]:
taime = (time.time() - t0)/60
xuleta.shemale(taime)

To:denismeia@icloud.com
From: denismeia@gmail.com
Subject:PROCESS DONE 

done!


** Polygon based **

In [ ]:
dates = [('2007-10-29'),('2008-01-09')]

In [ ]:
#POLYGON BASED TEST - BRASIL
SQL = ("""SELECT 
    geocodig_m , sigla, nome_munic, acquisition,
    (St_SummaryStats(St_Union(ST_Clip(rast,1,geom, true)))).*
FROM %s.%s , public.br_ne_municipios
WHERE st_intersects(rast,geom)
    AND acquisition BETWEEN '%s' AND '%s'
  --  AND sigla IN ('AL','BA','CE','MA','PB','PE','PI','SE','RN')
GROUP BY geocodig_m , sigla, nome_munic, acquisition
ORDER BY acquisition;
"""
       %(schema,table,dates[0],dates[1]))
print(SQL)
%sql $SQL